In [1]:
# Cell 1: Setup and Imports

import sys
import os
import pandas as pd
import torch

# --- Thêm đường dẫn project vào hệ thống ---
project_root = os.path.abspath(os.path.join(os.getcwd(), '..'))
if project_root not in sys.path:
    sys.path.append(project_root)


# Cấu hình để Pytorch hoạt động tốt với một số môi trường
os.environ["PYTORCH_ENABLE_MPS_FALLBACK"] = "1"
curr_dir = os.getcwd()
path = os.path.join(curr_dir, "..")



In [2]:
import dotenv

from nixtla import NixtlaClient

dotenv.load_dotenv()

nixtla_client = NixtlaClient(
    api_key=os.getenv("TimeGPT-API-Key")
)


nixtla_client.validate_api_key()

INFO:nixtla.nixtla_client:Happy Forecasting! :)


True

In [3]:
import pandas as pd

df = pd.read_csv(path + '/data/processed/data_normalized.csv')

df.rename(columns={'symbol': 'unique_id'}, inplace=True)

# df['unique_id'] = 'BTCUSDT'

df.head()

,timestamp,open,high,low,close,volume,unique_id,sentiment_score,close_lag_1,close_lag_7,ema_10,ema_20,volatility_10,rsi_14,day_of_week,day_sin,day_cos
0,2025-01-14,0.470736,0.498260,0.568333,0.564715,-0.744934,BTCUSDT,0.0,0.470738,0.834681,0.720497,0.868977,2714.238922,54.182724,1,0.781831,0.623490
1,2025-01-15,0.576382,0.750704,0.642522,0.833228,-0.486008,BTCUSDT,0.0,0.576384,0.766825,0.755351,0.882625,2989.433771,65.051060,2,0.974928,-0.222521
2,2025-01-16,0.847186,0.735632,0.739161,0.662687,-0.465569,BTCUSDT,0.0,0.847200,0.631049,0.750922,0.876345,2537.736889,62.966884,3,0.433884,-0.900969
3,2025-01-17,0.675182,0.811140,0.790559,0.859510,-0.329400,BTCUSDT,0.0,0.675196,0.691143,0.785322,0.892162,3608.518880,70.992879,4,-0.433884,-0.900969
4,2025-01-18,0.873717,0.774706,0.692186,0.662282,0.003757,BTCUSDT,0.0,0.873707,0.710977,0.775366,0.884930,4306.797050,71.764243,5,-0.974928,-0.222521


In [ ]:
from src.utils.normalizer import Normalizer

numeric_cols = ["open", "high", "low", "close", "volume", 
                "close_lag_1", "close_lag_7", "ema_10", "ema_20"]
norm = Normalizer("standard", per_symbol=True,columns=numeric_cols)
df_new = pd.read_csv(path + '/data/processed/data.csv')
df_new = norm.fit_transform(df_new)

fcst = nixtla_client.forecast(
    df=df,
    h=4,
    time_col='timestamp',
    target_col='close'
)


fcst.rename(columns={'unique_id': 'symbol', 'TimeGPT': 'close'}, inplace=True)

fcst.head()
res = norm.inverse_transform(fcst[['symbol', 'timestamp', 'close']])

res.head()

INFO:nixtla.nixtla_client:Validating inputs...
INFO:nixtla.nixtla_client:Inferred freq: D
/Users/quockhoile/Desktop/DaiHoc/KT/Project/AI-Predict/financial-prediction-system/services/ai-services/.venv/lib/python3.13/site-packages/nixtla/nixtla_client.py:380: UserWarning: `df` contains the following exogenous features: ['open', 'high', 'low', 'volume', 'sentiment_score', 'close_lag_1', 'close_lag_7', 'ema_10', 'ema_20', 'volatility_10', 'rsi_14', 'day_of_week', 'day_sin', 'day_cos'], but `X_df` was not provided and they were not declared in `hist_exog_list`. They will be ignored.
  warnings.warn(
INFO:nixtla.nixtla_client:Preprocessing dataframes...
INFO:nixtla.nixtla_client:Restricting input...
INFO:nixtla.nixtla_client:Calling Forecast Endpoint...


,symbol,timestamp,close
0,BTCUSDT,2025-09-04,124326.213107
1,BTCUSDT,2025-09-05,124343.125589
2,BTCUSDT,2025-09-06,124727.574602
3,BTCUSDT,2025-09-07,124768.011408
4,ETHUSDT,2025-09-04,4397.340249


In [10]:
df_1 = pd.read_csv(path + '/data/processed/data.csv')
df_1.rename(columns={'symbol': 'unique_id'}, inplace=True)
fcst = nixtla_client.forecast(
    df=df_1,
    h=4,
    time_col='timestamp',
    target_col='close'
)

fcst.head()

INFO:nixtla.nixtla_client:Validating inputs...
INFO:nixtla.nixtla_client:Inferred freq: D
/Users/quockhoile/Desktop/DaiHoc/KT/Project/AI-Predict/financial-prediction-system/services/ai-services/.venv/lib/python3.13/site-packages/nixtla/nixtla_client.py:380: UserWarning: `df` contains the following exogenous features: ['open', 'high', 'low', 'volume', 'sentiment_score', 'close_lag_1', 'close_lag_7', 'ema_10', 'ema_20', 'volatility_10', 'rsi_14', 'day_of_week', 'day_sin', 'day_cos'], but `X_df` was not provided and they were not declared in `hist_exog_list`. They will be ignored.
  warnings.warn(
INFO:nixtla.nixtla_client:Preprocessing dataframes...
INFO:nixtla.nixtla_client:Restricting input...
INFO:nixtla.nixtla_client:Calling Forecast Endpoint...


,unique_id,timestamp,TimeGPT
0,BTCUSDT,2025-09-04,111514.1250
1,BTCUSDT,2025-09-05,111107.0700
2,BTCUSDT,2025-09-06,110988.1700
3,BTCUSDT,2025-09-07,111040.8050
4,ETHUSDT,2025-09-04,4397.3403
